In [2]:
import pickle
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import h5py

In [3]:
root_dir = "/home/yyelisieiev/rsna_data/RSNA_ASNR_MICCAI_BraTS2021_TrainingData"
sample = "BraTS2021_00000"
sample_dir = "/home/yyelisieiev/rsna_data/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00000/"

In [16]:
def nib_load(file_name):
    proxy = nib.load(file_name)
    data = proxy.get_data()
    proxy.uncache()
    return data

In [17]:
modalities = ['flair', 't1ce', 't1', 't2']

In [7]:
all_cases = os.listdir('/home/yyelisieiev/rsna_data/RSNA_ASNR_MICCAI_BraTS2021_TrainingData')

In [10]:
all_cases[0]

'BraTS2021_00674'

In [29]:
a_file = open("val (1).txt", "r")

val_list = []
for line in a_file:
    stripped_line = line.strip()
#     line_list = stripped_line.split()
    list_of_lists.append(stripped_line)

a_file.close()

len(os.listdir('/home/yyelisieiev/rsna_data/preprocessed_BraTS/train'))

155155

In [33]:
train_dir = "/home/yyelisieiev/rsna_data/preprocessed_BraTS/train"

for case in tqdm(all_cases):
    
    print(case, val_list[0])
    if case in val_list:
        continue
    
    case_n = case.split("_")[1]
    sample_path = os.path.join(root_dir, case, case)
    images = np.stack([np.array(nib_load(sample_path + '_' + modal + '.nii.gz'), dtype='float32', order='C') for modal in modalities], -1)
    
    label = np.array(nib_load(sample_path + '_' + 'seg.nii.gz'), dtype='uint8', order='C')
    mask = images.sum(-1) > 0    
    
    for k in range(4):

        x = images[..., k]
        y = x[mask]
        x[mask] -= y.mean()
        x[mask] /= y.std()
        images[..., k] = x
    
    
    images = np.transpose(images, (2, 3, 0, 1))
    label = np.transpose(label, (2, 0, 1))
    
    for slice_idx, mri_slice in enumerate(zip(images, label)):
        image, label = mri_slice
        n_elements = label.size
        non_zeros = np.count_nonzero(label)
        
        non_zero_frac = non_zeros/n_elements
        
        if non_zero_frac > 0.01:
            slice_path = os.path.join(train_dir, f"case{case_n}_slice{slice_idx}.npz")
            np.savez(slice_path, image, label)

  0%|                                                                                                             | 0/1251 [00:00<?, ?it/s]/tmp/ipykernel_25856/3069981497.py:3: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data = proxy.get_data()
 45%|████████████████████████████████████████████▌                                                      | 563/1251 [09:26<11:31,  1.01s/it]


KeyboardInterrupt: 

In [ ]:
len(os.listdir('/home/yyelisieiev/rsna_data/preprocessed_BraTS/train'))

155155

In [ ]:
# val_dir = "/home/yyelisieiev/rsna_data/preprocessed_BraTS/val"

# for case in tval_listl_cases):
#     case_n = case.split("_")[1]
#     sample_path = os.path.join(root_dir, case, case)
#     images = np.stack([np.array(nib_load(sample_path + '_' + modal + '.nii.gz'), dtype='float32', order='C') for modal in modalities], -1)
#     label = np.array(nib_load(sample_path + '_' + 'seg.nii.gz'), dtype='uint8', order='C')
    
#     mask = images.sum(-1) > 0
    
#     for k in range(4):

#         x = images[..., k]
#         y = x[mask]
#         x[mask] -= y.mean()
#         x[mask] /= y.std()
#         images[..., k] = x
        
#     images = np.transpose(images, (2, 3, 0, 1))
#     label = np.transpose(label, (2, 0, 1))
    
#     f1 = h5py.File(os.path.join(val_dir, f"case{case_n}.npy.hdf5"), "w")
    
#     f1.create_dataset("images", data=images)
#     f1.create_dataset("label", data=label)

  0%|                                                                                                              | 0/250 [00:00<?, ?it/s]/tmp/ipykernel_85338/3069981497.py:3: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data = proxy.get_data()
 64%|████████████████████████████████████████████████████████████████                                    | 160/250 [03:27<02:13,  1.48s/it]

In [ ]:
with h5py.File("/home/yyelisieiev/rsna_data/preprocessed_BraTS/val/case00615.npy.hdf5", "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    data = list(f[a_group_key])